In [ ]:
import cv2
import os

img_dir = "data/images"
canny_dir = "data/canny"
os.makedirs(canny_dir, exist_ok=True)

for fname in os.listdir(img_dir):
    # print(fname)
    img = cv2.imread(os.path.join(img_dir, fname))
    # print(img)
    edges = cv2.Canny(img, 100, 200)
    cv2.imwrite(os.path.join(canny_dir, fname), edges)

Elets.png
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
Iasi.png
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
 

In [ ]:
accelerate launch train_controlnet_lora.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
  --controlnet_model_name_or_path="lllyasviel/sd-controlnet-canny" \
  --train_data_dir="data" \
  --output_dir="output" \
  --caption_file="captions.json" \
  --resolution=512 \
  --train_batch_size=5 \
  --learning_rate=1e-5 \
  --use_8bit_adam \
  --gradient_accumulation_steps=1 \
  --lr_scheduler="constant" \
  --num_train_epochs=100 \
  --validation_prompt="2 адм, 6 РВС 10к м, 1 РВС 500 м, 4 РВС 50 м, 2 жд" \
  --tracker_project_name="building-schemes"


^C


The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
C:\Users\nadyl\AppData\Local\Programs\Python\Python310\python.exe: can't open file 'c:\\Users\\nadyl\\Documents\\GitHub\\generation-of-building-diagrams\\Attempt 4\\train_controlnet_lora.py': [Errno 2] No such file or directory
Traceback (most recent call last):
  File "C:\Users\nadyl\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\nadyl\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\nadyl\Documents\GitHub\generation-of-building-diagr

Проверяем все описание на адекватное количество токенов, чтобы не потерять никакой информации

In [33]:
from transformers import CLIPTokenizer

tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")

text = "4 адм, 6 РВС 2к м, 8 ГВС 75 м, 8 ГВС 50 м, 2 жд"

tokens = tokenizer(text, truncation=False, padding=False)
num_tokens = len(tokens['input_ids'])

print(f"Количество токенов: {num_tokens}")

if num_tokens > 77:
    print("Текст слишком длинный и будет обрезан.")
else:
    print("Текст подходит по длине.")


Количество токенов: 46
Текст подходит по длине.


In [2]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
from diffusers import UniPCMultistepScheduler
from PIL import Image
import torch
import cv2

prompt = "Комната 5x5, дверь слева, окно справа"

controlnet = ControlNetModel.from_pretrained("output/adapter_config.json", torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16
)

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.to("cuda")

# Заготовка канни-карты (можно пустую)
canny_img = cv2.Canny(cv2.imread("data/images/plan1.png"), 100, 200)
canny_img = Image.fromarray(canny_img).convert("RGB")

image = pipe(prompt, num_inference_steps=25, image=canny_img).images[0]
image.save("result.png")


KeyError: 'weight_map'

In [3]:
from diffusers import StableDiffusionPipeline
import torch

# Загрузка базовой модели
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to("cuda")

# Добавление LoRA
pipe.unet.load_adapter(
    "output/adapter_config.json",  # Папка с adapter_config.json и adapter_model.bin
    adapter_name="building_lora"
)

# Генерация изображения
prompt = "2 адм, 6 РВС 10к м"
image = pipe(prompt, num_inference_steps=50).images[0]
image.save("result.png")

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

c:\Users\nadyl\Documents\GitHub\generation-of-building-diagrams\.venv\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nadyl\.cache\huggingface\hub\models--runwayml--stable-diffusion-v1-5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

AttributeError: 'UNet2DConditionModel' object has no attribute 'load_adapter'

In [4]:
from diffusers import StableDiffusionControlNetPipeline

controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny")
pipe = StableDiffusionControlNetPipeline(
    controlnet=controlnet
)

# Загрузка LoRA в UNet
pipe.unet.load_attn_procs("output/adapter_config.json")
canny_img = cv2.Canny(cv2.imread("data/images/plan1.png"), 100, 200)
canny_img = Image.fromarray(canny_img).convert("RGB")
# Использование
image = pipe(
    prompt="2 адм, 6 РВС 10к м",
    image=canny_img  # Ваше canny-изображение
).images[0]
image.save("result.png")

TypeError: StableDiffusionControlNetPipeline.__init__() missing 7 required positional arguments: 'vae', 'text_encoder', 'tokenizer', 'unet', 'scheduler', 'safety_checker', and 'feature_extractor'

In [5]:

import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from diffusers.utils import load_image
import cv2
from PIL import Image

# 1. Инициализация ControlNet
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-canny", 
    torch_dtype=torch.float16
)

# 2. Правильная инициализация пайплайна
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    torch_dtype=torch.float16,
    safety_checker=None,  # Отключаем для упрощения
    requires_safety_checker=False
).to("cuda")

# 3. Оптимизация производительности
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

# 4. Загрузка LoRA (должна быть папка с adapter_model.bin и adapter_config.json)
pipe.unet.load_attn_procs("output")  # Указываем папку, а не файл!

# 5. Подготовка canny-изображения
canny_img = cv2.Canny(cv2.imread("data/images/plan1.png"), 100, 200)
canny_img = Image.fromarray(canny_img).convert("RGB").resize((512, 512))

# 6. Генерация
image = pipe(
    prompt="2 адм, 6 РВС 10к м",
    image=canny_img,
    num_inference_steps=30,
    guidance_scale=7.5
).images[0]

image.save("result.png")

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

OSError: Error no file named pytorch_lora_weights.bin found in directory output.

In [3]:
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from diffusers.models.attention_processor import LoRAAttnProcessor
import cv2
from PIL import Image
import os
from safetensors.torch import load_file
import json

# ===== Конфигурация =====
BASE_MODEL = "runwayml/stable-diffusion-v1-5"
CONTROLNET_MODEL = "lllyasviel/sd-controlnet-canny"
LORA_PATH = "./output"  # Папка с adapter_config.json и adapter_model.safetensors
INPUT_IMAGE = "data/images/plan1.png"
PROMPT = "2 адм, 6 РВС 10к м"
OUTPUT_IMAGE = "result.png"

# ===== 1. Инициализация пайплайна =====
controlnet = ControlNetModel.from_pretrained(
    CONTROLNET_MODEL,
    torch_dtype=torch.float16
)

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    BASE_MODEL,
    controlnet=controlnet,
    torch_dtype=torch.float16,
    safety_checker=None,
    requires_safety_checker=False
).to("cuda")

# ===== 2. Загрузка LoRA весов (современный способ) =====
def load_lora_weights(pipeline, lora_path):
    # Проверяем наличие файлов
    if not os.path.exists(lora_path):
        raise ValueError(f"Папка {lora_path} не существует")
    
    # Определяем путь к весам
    weights_path = None
    for filename in ["adapter_model.safetensors", "adapter_model.bin"]:
        if os.path.exists(os.path.join(lora_path, filename)):
            weights_path = os.path.join(lora_path, filename)
            break
    
    if not weights_path:
        raise ValueError("Не найден файл с весами (adapter_model.safetensors или adapter_model.bin)")
    
    # Загружаем веса
    if weights_path.endswith(".safetensors"):
        state_dict = load_file(weights_path)
    else:
        state_dict = torch.load(weights_path, map_location="cpu")
    
    # Создаем процессоры
    unet = pipeline.unet
    attn_procs = {}
    
    # Получаем параметры из конфига
    config_path = os.path.join(lora_path, "adapter_config.json")
    if os.path.exists(config_path):
        with open(config_path, "r") as f:
            config = json.load(f)
        rank = config.get("r", 4)
        network_alpha = config.get("lora_alpha", None)
    else:
        rank = 4
        network_alpha = None
    
    # Инициализируем процессоры
    for name in unet.attn_processors.keys():
        cross_attention_dim = None if name.endswith("attn1.processor") else unet.config.cross_attention_dim
        attn_procs[name] = LoRAAttnProcessor(
            rank=rank,
            network_alpha=network_alpha
        )
    
    unet.set_attn_processor(attn_procs)
    
    # Загружаем веса
    for name, attn_processor in unet.attn_processors.items():
        if isinstance(attn_processor, LoRAAttnProcessor):
            layer_name = name.replace(".processor", "")
            attn_processor.load_state_dict({
                f"to_q_lora.up.weight": state_dict[f"{layer_name}.to_q_lora.up.weight"],
                f"to_k_lora.up.weight": state_dict[f"{layer_name}.to_k_lora.up.weight"],
                f"to_v_lora.up.weight": state_dict[f"{layer_name}.to_v_lora.up.weight"],
                f"to_out_lora.up.weight": state_dict[f"{layer_name}.to_out_lora.up.weight"],
            }, strict=False)

load_lora_weights(pipe, LORA_PATH)

# ===== 3. Оптимизация =====
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

# ===== 4. Подготовка входного изображения =====
def prepare_canny_image(image_path, low_threshold=100, high_threshold=200):
    image = cv2.imread(image_path)
    canny = cv2.Canny(image, low_threshold, high_threshold)
    canny = Image.fromarray(canny).convert("RGB").resize((512, 512))
    return canny

canny_image = prepare_canny_image(INPUT_IMAGE)

# ===== 5. Генерация изображения =====
result = pipe(
    prompt=PROMPT,
    image=canny_image,
    num_inference_steps=30,
    guidance_scale=7.5,
    generator=torch.Generator(device="cuda").manual_seed(42)
).images[0]

result.save(OUTPUT_IMAGE)
print(f"Изображение сохранено как {OUTPUT_IMAGE}")

# ===== Очистка =====
torch.cuda.empty_cache()

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

TypeError: LoRAAttnProcessor.__init__() got an unexpected keyword argument 'rank'

In [ ]:
python test_lora.py --input_image data/canny/Salsk.png --prompt "4 РВС 3к м, 4 РВС 1к м, 2 РВС 50 м, 5 ГВС 75 м, 6 ГВС 60 м"